# Análise da Motivação dos Alunos de Engenharia da Computação da Universidade Estadual do Maranhão (UEMA)

Análise exploratória dos dados da pesquisa sobre
a participação em laboratórios/projetos e motivação
dos alunos de Engenharia da Computação.

Dados:
- alunos_geral.csv
- bolsistas.csv

Objetivo:
Explorar, descrever e comparar percepções de alunos
participantes e não participantes de atividades extracurriculares.

Link repositório Github: https://github.com/Gabriel-c0Nsp/analise-motivacao

### Dependências

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

### Leitura dos CSVs

In [33]:
alunos = pd.read_csv("dados/alunos_geral.csv")
pesquisadores = pd.read_csv("dados/bolsistas.csv")

### Remoção de espaços em branco

In [34]:
alunos.columns = alunos.columns.str.strip()
pesquisadores.columns = pesquisadores.columns.str.strip()

### Funções auxiliares

### Setup

In [35]:
LIKERT_5 = {
    "Discordo totalmente": 1,
    "Discordo parcialmente": 2,
    "Nem concordo nem discordo": 3,
    "Neutro": 3,
    "Concordo parcialmente": 4,
    "Concordo totalmente": 5,
}

SIM_NAO = {"Não": 0, "Sim": 1}

COL = {
    # alunos (geral)
    "al_participa_lab": "Participa de algum laboratório ou projeto de pesquisa da universidade?",
    "al_lab_ajuda": "A participação em laboratórios ou projetos de pesquisa ajuda (ou ajudaria) a compreender melhor os conteúdos das disciplinas.",
    "al_fin_afeta": "As dificuldades financeiras afetam minha vida acadêmica.",
    "al_trocar_curso": "Considera trocar de curso?",
    "al_acolh_prof": "Me sinto acolhido(a) por professores e monitores quando tenho dúvidas.",

    # pesquisadores
    "pes_participacao": "Qual a sua participação em relação às atividades citadas acima?",
    "pes_falta_tempo": "Sinto que não tenho tempo suficiente para conciliar as disciplinas regulares, a atividade no laboratório/projeto e minha vida pessoal.",
    "pes_pensou_desistir": "Já pensou em desistir da atividade de laboratório ou extensão por excesso de demandas acadêmicas?",
    "pes_orientacao": "Recebo orientação adequada do meu professor orientador.",
    "pes_recomendaria": "Você já recomendou a colegas a participação no laboratório ou projeto?",
    "pes_reconhecimento_motiv": "O reconhecimento (formal ou informal) pelo meu trabalho no laboratório/projeto contribuiu para minha motivação no curso.",
    "pes_lab_ajuda": "A participação em projetos de pesquisa ou laboratórios ajudou a compreender melhor os conteúdos vistos em sala de aula.",
}

In [36]:
def clean_text(x):
    """Padroniza strings: remove espaços extras e converte para string quando possível."""
    if pd.isna(x):
        return x
    return str(x).strip()

def freq_table(df, col, dropna=False, sort_index=True):
    """
    Retorna uma tabela com contagem e percentual da coluna.
    - dropna=False mostra NaN
    """
    s = df[col].map(clean_text)
    counts = s.value_counts(dropna=dropna)
    if sort_index:
        counts = counts.sort_index()
    perc = (counts / counts.sum() * 100).round(1)
    return pd.DataFrame({"count": counts, "percent": perc})

## Uso ##
# col = "A participação em projetos de pesquisa ou laboratórios ajudou a compreender melhor os conteúdos vistos em sala de aula."
# freq_table(participantes, col)

def to_likert_num(df, col, new_col):
    """
    Cria new_col com valores 1..5 a partir de uma coluna Likert.
    Também retorna a contagem de NaN para você checar se algo não foi mapeado.
    """
    s = df[col].map(clean_text)
    df[new_col] = s.map(LIKERT_5)
    nan_count = df[new_col].isna().sum()
    return nan_count

## Uso ##
# col = "A participação em projetos de pesquisa ou laboratórios ajudou a compreender melhor os conteúdos vistos em sala de aula."
# nan = to_likert_num(participantes, col, "ajuda_compreender_num")
# nan

def to_bin_num(df, col, new_col):
    s = df[col].map(clean_text)
    df[new_col] = s.map(SIM_NAO)
    nan_count = df[new_col].isna().sum()
    return nan_count

## Uso ##
# col = "Participa de algum laboratório ou projeto de pesquisa da universidade?"
# nan = to_bin_num(alunos, col, "participa_lab")
# nan

def plot_likert_bar(df, col_num, title):
    counts = df[col_num].value_counts().sort_index()
    counts.plot(kind="bar")
    plt.xlabel("Likert (1=Discordo total, 5=Concordo total)")
    plt.ylabel("Número de respostas")
    plt.title(title)
    plt.show()

## Uso ##
# plot_likert_bar(participantes, "ajuda_compreender_num", "Participantes: ajuda a compreender conteúdos")

def crosstab_counts(df, a, b):
    """Tabela de contagens: A x B."""
    return pd.crosstab(df[a], df[b])

def crosstab_rowperc(df, a, b):
    """Tabela de percentuais por linha (cada linha soma 100%)."""
    tab = pd.crosstab(df[a], df[b])
    return (tab.div(tab.sum(axis=1), axis=0) * 100).round(1)

## Uso ##
# 1) garantir colunas numéricas
# to_bin_num(alunos, "Participa de algum laboratório ou projeto de pesquisa da universidade?", "participa_lab")
# to_likert_num(alunos, "A participação em laboratórios ou projetos de pesquisa ajuda (ou ajudaria) a compreender melhor os conteúdos das disciplinas.", "lab_ajuda_num")

# 2) cruzar
# crosstab_counts(alunos, "participa_lab", "lab_ajuda_num")
# crosstab_rowperc(alunos, "participa_lab", "lab_ajuda_num")

# Diferentes testes e análises exploratórias a partir daqui (temporário)

### Criação de colunas para análise

In [37]:
# Alunos: participa_lab × lab_ajuda_compreender
to_bin_num(alunos, COL["al_participa_lab"], "participa_lab")
to_likert_num(alunos, COL["al_lab_ajuda"], "lab_ajuda_num")

# Alunos: dificuldades_financeiras_afetam × considera_trocar_curso
to_likert_num(alunos, COL["al_fin_afeta"], "fin_afeta_num")
to_bin_num(alunos, COL["al_trocar_curso"], "trocar_curso")

# Alunos: acolhimento_professores × considera_trocar_curso
to_likert_num(alunos, COL["al_acolh_prof"], "acolh_prof_num")
# trocar_curso já criado acima

# Pesquisadores: falta_tempo_conciliar × pensou_desistir
to_likert_num(pesquisadores, COL["pes_falta_tempo"], "falta_tempo_num")
to_bin_num(pesquisadores, COL["pes_pensou_desistir"], "pensou_desistir")

# Pesquisadores: orientacao_orientador × recomendaria
to_likert_num(pesquisadores, COL["pes_orientacao"], "orientacao_num")
to_bin_num(pesquisadores, COL["pes_recomendaria"], "recomendaria")

# Pesquisadores: reconhecimento_motivacao × recomendaria
to_likert_num(pesquisadores, COL["pes_reconhecimento_motiv"], "reconhecimento_num")
# recomendaria já criado acima

# Pesquisadores: participacao (bolsista/voluntario) × pensou_desistir
pesquisadores["participacao"] = pesquisadores[COL["pes_participacao"]].map(clean_text)
# pensou_desistir já criado acima

# Ponte: comparar “laboratórios ajudam” entre formulários
to_likert_num(pesquisadores, COL["pes_lab_ajuda"], "lab_ajuda_num")

np.int64(0)

#### Alunos: participa_lab × lab_ajuda_num

In [38]:
display(freq_table(alunos, COL["al_participa_lab"]))
display(freq_table(alunos, COL["al_lab_ajuda"]))

display(crosstab_counts(alunos, "participa_lab", "lab_ajuda_num"))
display(crosstab_rowperc(alunos, "participa_lab", "lab_ajuda_num"))

,count,percent
Participa de algum laboratório ou projeto de pesquisa da universidade?,,
Não,18,62.1
Sim,11,37.9


,count,percent
A participação em laboratórios ou projetos de pesquisa ajuda (ou ajudaria) a compreender melhor os conteúdos das disciplinas.,,
Concordo parcialmente,12,41.4
Concordo totalmente,13,44.8
Discordo parcialmente,1,3.4
Discordo totalmente,1,3.4
Nem concordo nem discordo,2,6.9


lab_ajuda_num,1,2,3,4,5
participa_lab,,,,,
0,1,1,2,5,9
1,0,0,0,7,4


lab_ajuda_num,1,2,3,4,5
participa_lab,,,,,
0,5.6,5.6,11.1,27.8,50.0
1,0.0,0.0,0.0,63.6,36.4


#### Alunos: fin_afeta_num × trocar_curso

In [39]:
display(freq_table(alunos, COL["al_fin_afeta"]))
display(freq_table(alunos, COL["al_trocar_curso"]))

display(crosstab_counts(alunos, "trocar_curso", "fin_afeta_num"))
display(crosstab_rowperc(alunos, "trocar_curso", "fin_afeta_num"))

,count,percent
As dificuldades financeiras afetam minha vida acadêmica.,,
Concordo parcialmente,4,13.8
Concordo totalmente,3,10.3
Discordo parcialmente,6,20.7
Discordo totalmente,9,31.0
Nem concordo nem discordo,7,24.1


,count,percent
Considera trocar de curso?,,
Não,18,62.1
Sim,11,37.9


fin_afeta_num,1,2,3,4,5
trocar_curso,,,,,
0,3,5,6,2,2
1,6,1,1,2,1


fin_afeta_num,1,2,3,4,5
trocar_curso,,,,,
0,16.7,27.8,33.3,11.1,11.1
1,54.5,9.1,9.1,18.2,9.1


#### Alunos: acolh_prof_num × trocar_curso

In [40]:
display(freq_table(alunos, COL["al_acolh_prof"]))

display(crosstab_counts(alunos, "trocar_curso", "acolh_prof_num"))
display(crosstab_rowperc(alunos, "trocar_curso", "acolh_prof_num"))

,count,percent
Me sinto acolhido(a) por professores e monitores quando tenho dúvidas.,,
Concordo parcialmente,11,37.9
Concordo totalmente,4,13.8
Discordo parcialmente,4,13.8
Nem concordo nem discordo,10,34.5


acolh_prof_num,2,3,4,5
trocar_curso,,,,
0,2,6,7,3
1,2,4,4,1


acolh_prof_num,2,3,4,5
trocar_curso,,,,
0,11.1,33.3,38.9,16.7
1,18.2,36.4,36.4,9.1


#### Pesquisadores: falta_tempo_num × pensou_desistir

In [41]:
display(freq_table(pesquisadores, COL["pes_falta_tempo"]))
display(freq_table(pesquisadores, COL["pes_pensou_desistir"]))

display(crosstab_counts(pesquisadores, "pensou_desistir", "falta_tempo_num"))
display(crosstab_rowperc(pesquisadores, "pensou_desistir", "falta_tempo_num"))

,count,percent
"Sinto que não tenho tempo suficiente para conciliar as disciplinas regulares, a atividade no laboratório/projeto e minha vida pessoal.",,
Concordo parcialmente,13,52.0
Concordo totalmente,1,4.0
Discordo parcialmente,7,28.0
Discordo totalmente,1,4.0
Nem concordo nem discordo,3,12.0


,count,percent
Já pensou em desistir da atividade de laboratório ou extensão por excesso de demandas acadêmicas?,,
Não,13,52.0
Sim,12,48.0


falta_tempo_num,1,2,3,4,5
pensou_desistir,,,,,
0,1,5,1,6,0
1,0,2,2,7,1


falta_tempo_num,1,2,3,4,5
pensou_desistir,,,,,
0,7.7,38.5,7.7,46.2,0.0
1,0.0,16.7,16.7,58.3,8.3


#### Pesquisadores: orientacao_num × recomendaria

In [42]:
display(freq_table(pesquisadores, COL["pes_orientacao"]))
display(freq_table(pesquisadores, COL["pes_recomendaria"]))

display(crosstab_counts(pesquisadores, "recomendaria", "orientacao_num"))
display(crosstab_rowperc(pesquisadores, "recomendaria", "orientacao_num"))

,count,percent
Recebo orientação adequada do meu professor orientador.,,
Concordo parcialmente,6,24.0
Concordo totalmente,16,64.0
Discordo parcialmente,2,8.0
Nem concordo nem discordo,1,4.0


,count,percent
Você já recomendou a colegas a participação no laboratório ou projeto?,,
Não,4,16.0
Sim,21,84.0


orientacao_num,2,3,4,5
recomendaria,,,,
0,1,0,2,1
1,1,1,4,15


orientacao_num,2,3,4,5
recomendaria,,,,
0,25.0,0.0,50.0,25.0
1,4.8,4.8,19.0,71.4


#### Pesquisadores: reconhecimento_num × recomendaria

In [43]:
display(freq_table(pesquisadores, COL["pes_reconhecimento_motiv"]))

display(crosstab_counts(pesquisadores, "recomendaria", "reconhecimento_num"))
display(crosstab_rowperc(pesquisadores, "recomendaria", "reconhecimento_num"))

,count,percent
O reconhecimento (formal ou informal) pelo meu trabalho no laboratório/projeto contribuiu para minha motivação no curso.,,
Concordo parcialmente,7,28.0
Concordo totalmente,14,56.0
Discordo parcialmente,1,4.0
Nem concordo nem discordo,3,12.0


reconhecimento_num,2,3,4,5
recomendaria,,,,
0,0,2,1,1
1,1,1,6,13


reconhecimento_num,2,3,4,5
recomendaria,,,,
0,0.0,50.0,25.0,25.0
1,4.8,4.8,28.6,61.9


#### Pesquisadores: participacao × pensou_desistir

In [44]:
display(freq_table(pesquisadores, COL["pes_participacao"]))
display(freq_table(pesquisadores, COL["pes_pensou_desistir"]))

display(crosstab_counts(pesquisadores, "participacao", "pensou_desistir"))
display(crosstab_rowperc(pesquisadores, "participacao", "pensou_desistir"))

,count,percent
Qual a sua participação em relação às atividades citadas acima?,,
Bolsista,16,64.0
Não se aplica à minha atividade,3,12.0
Voluntário,6,24.0


,count,percent
Já pensou em desistir da atividade de laboratório ou extensão por excesso de demandas acadêmicas?,,
Não,13,52.0
Sim,12,48.0


pensou_desistir,0,1
participacao,,
Bolsista,11,5
Não se aplica à minha atividade,1,2
Voluntário,1,5


pensou_desistir,0,1
participacao,,
Bolsista,68.8,31.2
Não se aplica à minha atividade,33.3,66.7
Voluntário,16.7,83.3


#### Ponte: comparar “lab_ajuda” entre alunos e pesquisadores

In [45]:
# alunos
al_counts = alunos["lab_ajuda_num"].value_counts().sort_index()
al_perc = (al_counts / al_counts.sum() * 100).round(1)

# pesquisadores
pes_counts = pesquisadores["lab_ajuda_num"].value_counts().sort_index()
pes_perc = (pes_counts / pes_counts.sum() * 100).round(1)

pd.DataFrame({
    "alunos_count": al_counts,
    "alunos_%": al_perc,
    "pesq_count": pes_counts,
    "pesq_%": pes_perc,
}).fillna(0)

,alunos_count,alunos_%,pesq_count,pesq_%
lab_ajuda_num,,,,
1,1,3.4,0.0,0.0
2,1,3.4,1.0,4.0
3,2,6.9,4.0,16.0
4,12,41.4,9.0,36.0
5,13,44.8,11.0,44.0
